In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from auxiliary import values as v
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar

from nuclei_segmentation import my_cellpose
from nuclei_segmentation import preprocessing
from nuclei_segmentation.quality_control.model_tester import ModelTester

GPU activated: False


In [2]:
v.data_path = '/mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/'

img_path = (
        v.data_path 
        + 'Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz'
)

Grid search into preprocessing steps 

In [14]:
from itertools import permutations, chain


# Cellpose params
steps_type = ['2D', '3D']
steps_thr = list(np.round(np.linspace(0.2, .7, 6), 1))

# Preprocessing steps
normalization_methods = ['norm_minmax', 'norm_percentile']
pre_steps_top = ['isotropy']
pre_steps_bottom = [
    'bilateral', 
    'anisodiff'
]

pre_steps_permutations = list(chain(*[
    permutations(pre_steps_bottom, r) 
    for r in range(1, len(pre_steps_bottom) + 1)
]))

# Postprocessing steps
post_steps = [
    'remove_small_objects',
    '3d_connected_component_analysis',
    'watershed'
]

post_steps_permutations = list(chain(*[
    permutations(post_steps, r) 
    for r in range(1, len(post_steps) + 1)
])) + [()]

pipelines_dict = {}

# Create pipelines for each type (2D/3D)
for step in steps_type:
    # Apply threshold only for 2D
    if step == '2D':
        for thr in steps_thr:
            for norm_method in normalization_methods:
                for i, pre_steps in enumerate(pre_steps_permutations):
                    for j, post_steps in enumerate(post_steps_permutations):
                        pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                        pipeline = pipeline_pre + list(post_steps)
                        
                        # Include threshold for 2D pipelines
                        name = f"pipeline_{step}_thr_{thr}_{norm_method}_{i}_{j}"
                        pipelines_dict[name] = {
                            'pipeline': pipeline,
                            'type': step,
                            'threshold': thr
                        }
    else:  # For 3D pipelines, ignore thresholds
        for norm_method in normalization_methods:
            for i, pre_steps in enumerate(pre_steps_permutations):
                for j, post_steps in enumerate(post_steps_permutations):
                    pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                    pipeline = pipeline_pre + list(post_steps)
                    
                    # No threshold for 3D pipelines
                    name = f"pipeline_{step}_{norm_method}_{i}_{j}"
                    pipelines_dict[name] = {
                        'pipeline': pipeline,
                        'type': step
                    }

print('Total pipelines:', len(pipelines_dict))

Total pipelines: 896


In [15]:
# Print all pipelines
for name, config in pipelines_dict.items():
    print(name, config)

pipeline_2D_thr_0.2_norm_minmax_0_0 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', 'remove_small_objects'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_1 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', '3d_connected_component_analysis'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_2 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', 'watershed'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_3 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', 'remove_small_objects', '3d_connected_component_analysis'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_4 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', 'remove_small_objects', 'watershed'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_5 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', '3d_connected_component_analysis', 'remove_small_objects'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_6

In [5]:
model = my_cellpose.load_model('nuclei')
tester = ModelTester(model)

Loading model: nuclei


In [6]:
bar = LoadingBar(len(pipelines_dict))

for name, config in pipelines_dict.items():
    print(f"\nTesting {name} pipeline")
    
    pipeline = config['pipeline']
    type = config['type']
    thr = config['threshold']
    
    tester.run(
        img_path, pipeline,
        type=type, stitch_threshold=thr,
        test_name=name, 
        verbose=0
    )
    
    bar.update()
    
bar.end()

Testing pipeline_2D_0_thr_0.1 pipeline
Running test: pipeline_2D_0_thr_0.1
Reading NIfTI: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz

Saving prediction: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_2D_0_thr_0.1_filtered.nii.gz


100%|██████████| 19/19 [00:00<00:00, 32.18it/s]

Masks shape: (20, 110, 140)

Saving prediction: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_2D_0_thr_0.1.nii.gz
[=                                                 ] 3.12%

Testing pipeline_2D_1_thr_0.1 pipeline
Running test: pipeline_2D_1_thr_0.1
Reading NIfTI: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz

Saving prediction: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_2D_1_thr_0.1_filtered.nii.gz


100%|██████████| 19/19 [00:00<00:00, 6738.69it/s]

...

Masks shape: (20, 110, 140)

Saving prediction: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_2D_0_thr_0.8.nii.gz
[=======================                           ] 46.88%

Testing pipeline_2D_1_thr_0.8 pipeline
Running test: pipeline_2D_1_thr_0.8
Reading NIfTI: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz

Saving prediction: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_2D_1_thr_0.8_filtered.nii.gz


100%|██████████| 19/19 [00:00<00:00, 4583.68it/s]

Masks shape: (20, 110, 140)

Saving prediction: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_2D_1_thr_0.8.nii.gz
[=========================                         ] 50.00%

Testing pipeline_3D_0_thr_0.1 pipeline
Running test: pipeline_3D_0_thr_0.1
Reading NIfTI: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz

Saving prediction: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_3D_0_thr_0.1_filtered.nii.gz
Masks shape: (20, 110, 140)

Saving prediction: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_3D_0_thr_0.1.nii.gz
[==========================                        ] 53.12%Testing pipeline_3D_1_thr_0.1 pipeline
Running test: pipeline_3D_1_thr_0.1
Reading NIfTI: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz

Saving prediction: /mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_3D_1_thr_0.1_filtered.nii.gz
Masks 